In [1]:
!pip install -q langchain_core langchain-openai langgraph langchain_community langchain_pinecone

In [2]:
%cd /content/drive/MyDrive/Agent/mangrove_agent

/content/drive/MyDrive/Agent/mangrove_agent


In [3]:
# Colab-specific secret storage
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = 'fe362b92-0b60-4a2d-877b-cd905d3f64f3'
os.environ["PINECONE_INDEX_NAME"] = 'mangrove-index'

In [4]:
# LLM
from langchain_openai import ChatOpenAI
from google.colab import userdata
llm = ChatOpenAI(model="gpt-4o", api_key=userdata.get('OPENAI_API_KEY'), temperature=0.8)

In [5]:
from pydantic import BaseModel, Field
from typing import Optional, Literal

class QueryIntent(BaseModel):
    goal: Literal["forecast", "research"] = Field(
        description="""
        'forecast': the user wants a contextual overview or analysis. May include NDVI forecasts and real-time data if a location is mentioned.
        'research': the user is asking a general knowledge question not tied to a specific place or data fetch.
        """
    )
    location: Optional[str] = Field(
        description="The geographic location specified in the query, if any. Return None if not found."
    )
    state: Optional[str] = Field(
    description="""
    The 2-letter U.S. state abbreviation (e.g., 'FL', 'TX') associated with the location,
    if known or inferable from the query. Helps scope station lookups regionally.
    Return None if the query doesn't reference a U.S. state.
    """
)

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(content="""
You are an intelligent assistant that extracts user intent from natural language.

Return:
- goal:
    - 'forecast' if the user is asking for an overview, condition, or analysis of mangroves, especially in a specific location.
    - 'research' if the user asks general questions like "What are mangroves?" or "Why are they important?"

- location:
    - If the user specifies a geographic location (e.g., city, region, island, or landmark), extract it.
    - Return None if no clear location is mentioned.

- state:
    - If the location is in the United States, extract the 2-letter state abbreviation (e.g., "FL" for Florida, "LA" for Louisiana).
    - Return None if the location is outside the U.S. or cannot be inferred.

Only use the allowed values for 'goal'. Do not guess location if it's unclear.
""")

structured_llm = llm.with_structured_output(QueryIntent)

In [7]:
def extract_intent_node(state: dict) -> dict:
    user_query = state["user_query"]

    intent = structured_llm.invoke([system_prompt, user_query])

    return {
        "goal": intent.goal,
        "location": intent.location,
        "state": intent.state
    }


In [8]:
import requests

def geocode_location(location_name: str):
    url = f"https://nominatim.openstreetmap.org/search?q={location_name}&format=json"
    r = requests.get(url, headers={"User-Agent": "LangGraph-Mangrove-Agent"})
    data = r.json()
    if not data:
        raise ValueError(f"Could not geocode location: {location_name}")
    lat = float(data[0]["lat"])
    lon = float(data[0]["lon"])
    return (lat, lon)


# Forecast

In [9]:
import pandas as pd

def load_stations(csv_path: str, state_abbr: str) -> list[dict]:
    df = pd.read_csv(csv_path)

    # Ensure state column exists and is properly formatted
    if "state" not in df.columns:
        raise ValueError("Missing 'state' column in station CSV.")

    if not state_abbr:  # 💡 Fix: don't attempt upper() on None
        return []

    filtered = df[df["state"].str.upper() == state_abbr.upper()]
    return filtered.to_dict(orient="records")  # for LLM ranking


In [10]:
def ask_llm_rank_stations(location: str, station_list: list[dict], variable: str) -> list[dict]:
    station_names = [s['name'] for s in station_list]

    prompt = f"""
    The user is asking about: {location}
    Sensor type: {variable}

    Here are some stations in the same U.S. state:
    {station_names}

    Rank the top 5 stations that are most relevant or closest to the location. Respond with a list of names (copy them exactly from the list).
    """

    response = llm.invoke(prompt).content
    print(f"=== Top 5 Nearest {variable.title()} Station ===") # Raw LLM Output
    print(response)
    print()

    # Normalize and remove list numbering (e.g., "1. Station Name")
    import re
    ranked_names = [
        re.sub(r"^\d+\.\s*", "", name.strip().lower())
        for name in response.split("\n")
        if name.strip()
    ]

    matched = []
    for ranked_name in ranked_names:
        for s in station_list:
            if ranked_name in s["name"].lower():
                matched.append(s)
                break
    return matched[:5]


In [11]:
def select_station_by_location_node(state: dict) -> dict:
    location = state.get("location")
    state_abbr = state.get("state")

    # if not location or not state_abbr:
    #     # Fallback to default station IDs
    #     return {
    #         "station_ids": {
    #             "wind": "42020",
    #             "water": "8723970"
    #         }
    #     }

    # Filter station lists by state
    wind_stations = load_stations("wind_speed_stations.csv", state_abbr)
    water_stations = load_stations("water_level_stations.csv", state_abbr)

    # Exit early if state is not in CSV = nothing to process = avoid indexing
    if not wind_stations and not water_stations:
        return {
            "station_ids": {},
            "station_candidates": {},
            "data_available": []
        }

    # Ask LLM to rank stations by proximity to 'location'
    ranked_wind = ask_llm_rank_stations(location, wind_stations, "wind")
    ranked_water = ask_llm_rank_stations(location, water_stations, "water level")

    return {
        "station_ids": {
            "wind": ranked_wind[0]["station_id"],
            "water": ranked_water[0]["station_id"]
        },
        "station_candidates": {
            "wind": [s["station_id"] for s in ranked_wind],
            "water": [s["station_id"] for s in ranked_water]
        }
    }

In [12]:
def fetch_wind_speed(station_id: str) -> Optional[float]:
    url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
    params = {
        "date": "latest",
        "station": station_id,
        "product": "wind",
        "units": "english",
        "time_zone": "gmt",
        "format": "json"
    }

    try:
        r = requests.get(url, params=params, timeout=5)
        if not r.ok:
            print(f"[{station_id}] Bad response: {r.status_code}")
            return None

        data = r.json()
        if "data" not in data or not data["data"]:
            print(f"[{station_id}] No 'data' in wind response")
            return None

        wind_speed = float(data["data"][0]["s"])
        return wind_speed
    except Exception as e:
        print(f"[{station_id}] Wind speed fetch error: {e}")
        return None


In [13]:
def fetch_water_level(station_id: str) -> Optional[float]:
    url = "https://api.tidesandcurrents.noaa.gov/api/prod/datagetter"
    params = {
        "date": "latest",
        "station": station_id,
        "product": "water_level",
        "datum": "MLLW",
        "units": "english",
        "time_zone": "gmt",
        "format": "json"
    }

    try:
        r = requests.get(url, params=params, timeout=5)
        r.raise_for_status()
        data = r.json()
        if "data" in data and data["data"]:
            return float(data["data"][0]["v"])
        print(f"[{station_id}] No data in response.")
        return None
    except Exception as e:
        print(f"[{station_id}] Water level fetch error: {e}")
        return None


In [14]:
def try_fetch_with_fallback(fetch_func, candidate_ids: list[str]) -> Optional[float]:
    for station_id in candidate_ids:
        result = fetch_func(station_id)
        if result is not None:
            return result
    return None


In [15]:
def fetch_environmental_data_node(state: dict) -> dict:
    stations = state.get("station_ids", {})
    candidates = state.get("station_candidates", {})

    wind_ids = candidates.get("wind", [stations.get("wind")]) if "wind" in stations else []
    water_ids = candidates.get("water", [stations.get("water")]) if "water" in stations else []

    wind = try_fetch_with_fallback(fetch_wind_speed, wind_ids) if wind_ids else None
    water = try_fetch_with_fallback(fetch_water_level, water_ids) if water_ids else None

    return {
        "environmental_data": {
            "wind_speed": wind,
            "water_level": water
        }
    }


In [16]:
import ee
import datetime

ee.Authenticate()

# Initialize with your project
ee.Initialize(project='ee-lgharijanto123')

def get_cleaned_weekly_ndvi_series(lat: float, lon: float, days_back: int = 70) -> list[float]:
    point = ee.Geometry.Point([lon, lat])
    study_area = point.buffer(16000)

    today = datetime.date.today()
    start = ee.Date(today.strftime('%Y-%m-%d')).advance(-days_back, 'day')

    def add_ndvi(image):
        ndvi = image.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')
        return image.addBands(ndvi)

    def extract_mean(image):
        mean = image.select('NDVI').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=study_area,
            scale=500,
            maxPixels=1e9,
            bestEffort=True
        ).get('NDVI')
        return ee.Feature(None, {'ndvi': mean})

    ndvi_series = (
        ee.ImageCollection('MODIS/061/MOD09GA')
        .filterBounds(study_area)
        .filterDate(start, ee.Date(today.strftime('%Y-%m-%d')))
        .map(add_ndvi)
        .map(extract_mean)
        .filter(ee.Filter.notNull(['ndvi']))
    )

    ndvi_list = ndvi_series.aggregate_array('ndvi').getInfo()
    ndvi_floats = [v / 10000 if v > 1 else v for v in ndvi_list]

    if len(ndvi_floats) < 56:
        raise ValueError(f"Not enough data. Got {len(ndvi_floats)} daily values, need at least 56.")

    weekly_ndvi = []
    for i in range(0, len(ndvi_floats) - 56 + 56, 7):
        chunk = ndvi_floats[i:i+7]
        if len(chunk) == 7:
            weekly_ndvi.append(sum(chunk) / 7)

    if len(weekly_ndvi) < 8:
        raise ValueError(f"Only {len(weekly_ndvi)} weekly values found, need 8.")

    return weekly_ndvi[-8:]


In [17]:
def fetch_weekly_noaa_lags_chunked(station_id: str, product: str, total_days: int = 56) -> list[float]:
    from datetime import datetime, timedelta
    import requests
    import pandas as pd

    end = datetime.utcnow().date()
    start = end - timedelta(days=total_days)

    # Break range into 3 chunks (e.g., 18 + 19 + 19 days)
    chunk_starts = [start + timedelta(days=i * 18) for i in range(3)]
    chunk_ends = [min(s + timedelta(days=18), end) for s in chunk_starts]

    all_dfs = []

    for chunk_start, chunk_end in zip(chunk_starts, chunk_ends):
        params = {
            "begin_date": chunk_start.strftime("%Y%m%d"),
            "end_date": chunk_end.strftime("%Y%m%d"),
            "station": station_id,
            "product": product,
            "datum": "MLLW" if product == "water_level" else None,
            "interval": "h",  # Hourly granularity
            "units": "english",
            "time_zone": "gmt",
            "format": "json"
        }

        try:
            r = requests.get("https://api.tidesandcurrents.noaa.gov/api/prod/datagetter",
                             params={k: v for k, v in params.items() if v is not None},
                             timeout=15)
            r.raise_for_status()
            data = r.json().get("data", [])
            if not data:
                continue

            df = pd.DataFrame(data)
            df["t"] = pd.to_datetime(df["t"])
            df.set_index("t", inplace=True)

            value_col = "s" if product == "wind" else "v"
            df[value_col] = pd.to_numeric(df[value_col], errors="coerce")

            all_dfs.append(df)

        except Exception as e:
            print(f"Chunk {chunk_start}–{chunk_end} failed: {e}")

    if not all_dfs:
        print(f"No valid data collected for {product}.")
        return []

    full_df = pd.concat(all_dfs).sort_index()

    # Resample into weekly means (week ends Sunday by default)
    value_col = "s" if product == "wind" else "v"
    weekly = full_df[value_col].resample("W").mean().dropna()

    if len(weekly) < 8:
        print(f"Only {len(weekly)} weekly values found for {product}, expected 8.")
        return weekly.tolist()  # Return whatever we have

    return weekly.tail(8).tolist()


In [18]:
import pandas as pd

def build_feature_vector_node(state: dict) -> dict:
    try:
        # Fetch 8 weeks (current + 7 lags)
        wind_vals = fetch_weekly_noaa_lags_chunked(state["station_ids"]["wind"], "wind")
        water_vals = fetch_weekly_noaa_lags_chunked(state["station_ids"]["water"], "water_level")
        ndvi_vals = get_cleaned_weekly_ndvi_series(*state["gps"])

        if len(wind_vals) < 8 or len(water_vals) < 8 or len(ndvi_vals) < 8:
            print("Insufficient weekly data.")
            return {"feature_vector": [], "feature_df": None}

        # Build DataFrame (already in chronological order: oldest → newest)
        df = pd.DataFrame({
            "tide_verified": water_vals,
            "wind_speed": wind_vals,
            "ndvi": ndvi_vals
        })

        # Add artificial weekly dates (most recent week = today)
        base_date = pd.to_datetime("today").normalize()
        df["date"] = [base_date - pd.Timedelta(weeks=i) for i in reversed(range(len(df)))]
        df.set_index("date", inplace=True)

        # Add lag features (1–7)
        for col in ["tide_verified", "wind_speed", "ndvi"]:
            for lag in range(1, 8):
                df[f"{col}_lag_{lag}"] = df[col].shift(lag)

        # Drop NaNs → only final row will be complete
        latest_row = df.dropna().iloc[-1]

        # Select 21 lag features in model training order
        # Get current values first
        features = latest_row[["tide_verified", "wind_speed"]].tolist()

        # Add lags
        features += latest_row[
            [f"{col}_lag_{i}" for col in ["tide_verified", "wind_speed", "ndvi"] for i in range(1, 8)]
        ].tolist()

        print("[feature vector]", features)

        return {
            "feature_vector": features,
            "feature_df": df
        }

    except Exception as e:
        print("Feature vector error:", e)
        return {"feature_vector": [], "feature_df": None}


In [19]:
import joblib
import numpy as np

# Load once at the top level (recommended)
xgb_model = joblib.load("xgboost.pkl")
scaler = joblib.load("scaler.pkl")

def predict_ndvi_node(state: dict) -> dict:
    try:
        features = state.get("feature_vector")
        if not features or len(features) != 23:
            print("Invalid or missing feature vector")
            return {"ndvi_prediction": None}

        X = pd.DataFrame([features], columns=scaler.feature_names_in_)

        X_scaled = scaler.transform(X)

        pred = xgb_model.predict(X_scaled)[0]
        print("NDVI Prediction:", pred)

        return {"ndvi_prediction": float(pred)}

    except Exception as e:
        print("Prediction error:", e)
        return {"ndvi_prediction": None}


In [20]:
from typing import TypedDict, Optional, Tuple, Dict, List

class State(TypedDict):
    # === User input & intent extraction ===
    user_query: str                             # Always provided by the user
    goal: Optional[str]                         # 'forecast' or 'research'
    location: Optional[str]                     # e.g. "Key West"
    state: Optional[str]                        # e.g. "FL"

    # === Location resolution ===
    gps: Optional[Tuple[float, float]]          # (lat, lon)

    # === Station selection ===
    station_ids: Optional[Dict[str, str]]       # {wind: id, water: id}
    station_candidates: Optional[Dict[str, List[str]]]  # fallback ids

    # === Real-time fetches
    environmental_data: Optional[Dict[str, float]]  # {'wind_speed': val, 'water_level': val}

    # === Weekly history (optional: for inspection only)
    wind_lags: Optional[List[float]]
    water_lags: Optional[List[float]]
    ndvi_lags: Optional[List[float]]


    # === Feature engineering ===
    feature_vector: Optional[List[float]]       # Final model input
    feature_df: Optional["pd.DataFrame"]        # Optional for debugging

    # === Model output ===
    ndvi_prediction: Optional[float]
    summary: Optional[str]
    final_output: Optional[str]

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable

summary_prompt = ChatPromptTemplate.from_template("""
You are a coastal ecology assistant.

Given the following data:
- User question: {user_query}
- NDVI prediction: {ndvi_prediction}
- Wind speed: {wind_speed}
- Water level: {water_level}

Generate a clear, concise 5-6 sentence summary of the mangrove condition and any notable environmental trends.
- Always include all the numbers IF provided.
- **DO NOT MENTION MISSING OR UNAVAILABLE DATA**
- If the query depends on geographic specificity and no location was found, consider asking the user to clarify a region.
""")

summary_chain: Runnable = summary_prompt | llm


def generate_summary_node(state: State) -> dict:
    try:
        response = summary_chain.invoke({
            "user_query": state["user_query"],
            "ndvi_prediction": state["ndvi_prediction"],
            "wind_speed": state["environmental_data"]["wind_speed"],
            "water_level": state["environmental_data"]["water_level"]
        })

        return {
            "summary": response.content
        }

    except Exception as e:
        print("[summary] Error:", e)
        return {"summary": "Unable to generate summary."}


In [22]:
def resolve_gps_from_location_node(state: dict) -> dict:
    location = state.get("location")
    state_abbr = state.get("state")

    print(f"[resolve_gps] location={location}, state={state_abbr}")

    if not location:
        print("[resolve_gps] Missing location.")
        return {"gps": None}

    try:
        query = location if not state_abbr else f"{location}, {state_abbr}"
        lat, lon = geocode_location(query)
        print(f"[resolve_gps] Geocoded → {lat}, {lon}")
        return {"gps": (lat, lon)}
    except Exception as e:
        print("[resolve_gps] Error:", e)
        return {"gps": None}


In [23]:
def fetch_ndvi_lags_node(state: dict) -> dict:
    gps = state.get("gps")
    if not gps:
        print("[ndvi lags] GPS missing")
        return {"ndvi_lags": []}
    lat, lon = gps
    ndvi_lags = get_cleaned_weekly_ndvi_series(lat, lon)[:7]
    print("[ndvi lags]", ndvi_lags)
    return {"ndvi_lags": ndvi_lags}


In [24]:
def fetch_weekly_lags_node(state: dict) -> dict:
    stations = state.get("station_ids", {})
    wind = fetch_weekly_noaa_lags_chunked(stations.get("wind"), "wind")
    water = fetch_weekly_noaa_lags_chunked(stations.get("water"), "water_level")
    print("[wind lags]", wind)
    print("[water lags]", water)
    return {"wind_lags": wind, "water_lags": water}


In [25]:
from typing import Literal

def route_by_goal(state: dict) -> Literal["forecast", "research"]:
    if state.get("goal") == "forecast":
        return "forecast"
    return "research"

In [26]:
from langgraph.graph import StateGraph, END

# Initialize graph
workflow = StateGraph(State)

# --- Routing Entry ---
workflow.add_node("extract_intent", extract_intent_node)
workflow.add_node("exit_early", lambda state: {"final_output": "[ROUTED TO RESEARCH AGENT]"})

workflow.set_entry_point("extract_intent")
workflow.add_conditional_edges("extract_intent", route_by_goal, {
    "forecast": "select_stations",
    "research": "exit_early"
})

# --- Parallel metadata fetch ---
workflow.add_node("select_stations", select_station_by_location_node)
workflow.add_node("resolve_gps", resolve_gps_from_location_node)
workflow.add_node("fetch_environmental_data", fetch_environmental_data_node)
workflow.add_node("fetch_weekly_lags", fetch_weekly_lags_node)

workflow.add_edge("select_stations", "resolve_gps")
workflow.add_edge("select_stations", "fetch_environmental_data")
workflow.add_edge("select_stations", "fetch_weekly_lags")

# --- Parallel NDVI + Weekly Lags ---
workflow.add_node("fetch_ndvi_lags", fetch_ndvi_lags_node)
workflow.add_edge("resolve_gps", "fetch_ndvi_lags")

# --- Feature Engineering + Prediction ---
workflow.add_node("build_feature_vector", build_feature_vector_node)
workflow.add_node("predict_ndvi", predict_ndvi_node)
workflow.add_node("generate_summary", generate_summary_node)

workflow.add_edge("fetch_ndvi_lags", "build_feature_vector")
workflow.add_edge("fetch_weekly_lags", "build_feature_vector")
workflow.add_edge("build_feature_vector", "predict_ndvi")
workflow.add_edge("predict_ndvi", "generate_summary")
workflow.add_edge("generate_summary", END)

# Compile
app = workflow.compile()


In [27]:
result = app.invoke({"user_query": "How are mangroves doing in Key West?"})

=== Top 5 Nearest Wind Station ===
1. Key West, FL
2. Vaca Key, Florida Bay, FL
3. Fort Myers, FL
4. Navy Fuel Depot, FL
5. Virginia Key, FL

=== Top 5 Nearest Water Level Station ===
1. Key West, FL
2. Vaca Key, Florida Bay, FL
3. Virginia Key, FL
4. South Port Everglades, FL
5. Naples Bay, North, FL

[resolve_gps] location=Key West, state=FL
[resolve_gps] Geocoded → 24.5548262, -81.8020722
[wind lags] [8.817380952380953, 9.12484375, 8.936190476190475, 8.84607142857143, 8.778385416666667, 8.63452380952381, 11.011666666666667, 10.238333333333333]
[water lags] [0.8182613095238096, 1.2668296875, 1.1934892857142856, 1.0953577380952382, 0.9231151041666668, 1.023640476190476, 1.0700666666666667, 0.9236666666666667]
[ndvi lags] [-0.2825340913753243, -0.3797324490112777, -0.21891654672869104, -0.12425821835979202, -0.35854588817151856, -0.29455943701618353, -0.34007538183998404]
[feature vector] [0.9236666666666667, 10.238333333333333, 1.0700666666666667, 1.023640476190476, 0.9231151041666668

In [28]:
print(result['summary'])

Mangroves in Key West appear to be in moderate condition, as suggested by the NDVI (Normalized Difference Vegetation Index) prediction of 0.4308. This value indicates a healthy, but not optimal, level of vegetation density and vigor. The current wind speed of 15.16 mph is typical for coastal regions but can influence mangrove stability and health, especially during storm events. Additionally, the water level is at 1.569 meters, which is within a range that generally supports mangrove ecosystems by providing necessary tidal flows and nutrient exchanges. Overall, while the environmental conditions support mangrove health, monitoring is essential to ensure that these critical ecosystems remain resilient.


# LangChain

In [29]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import Pinecone

# Setup memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Setup retriever
embeddings = OpenAIEmbeddings()
db = Pinecone.from_existing_index(index_name="mangrove-index", embedding=embeddings)
retriever = db.as_retriever()

# Research chain
research_chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

<ipython-input-29-0a31e7ae05c5>:7: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [30]:
def forecast_chain(user_query: str) -> str:
    result = app.invoke({"user_query": user_query})
    return result.get("summary", "No forecast available.")


In [31]:
def run_research_chain(user_query: str) -> str:
    raw_response = research_chain.invoke({"question": user_query})
    return raw_response.get("answer") if isinstance(raw_response, dict) else str(raw_response)


In [32]:
structured_llm = llm.with_structured_output(QueryIntent)

In [33]:
def extract_goal_and_location(user_query: str):
    intent = structured_llm.invoke([system_prompt, user_query])
    return intent.goal


In [34]:
def run_agent(user_query: str):
    print(f"🧪 User query: {user_query}")

    goal = extract_goal_and_location(user_query)
    print(f"🔍 Detected goal: {goal}")

    if goal == "forecast":
        final_output = forecast_chain(user_query)

        memory.save_context({"input": user_query}, {"output": final_output})

    elif goal == "research":
        final_output = run_research_chain(user_query)

    else:
        final_output = "Unrecognized goal."

    return {"final_output": final_output}


In [35]:
def print_chat_history():
    print("🧠 Chat History:")
    for msg in memory.chat_memory.messages:
        role = "👤 User" if msg.type == "human" else "🤖 Assistant"
        print(f"{role}: {msg.content}\n")


In [36]:
memory.clear()

In [37]:
response = run_agent("What are mangroves?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("How are mangroves doing in Florida?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

print_chat_history()


🧪 User query: What are mangroves?
🔍 Detected goal: research
🤖 LLM Answer: Mangroves are a taxonomically diverse group of approximately 70 tree, shrub, and fern species that grow in anoxic (oxygen-poor) and saline peaty soils on sheltered, tropical coasts. These species belong to at least 25 genera and 19 families. Mangroves are well adapted to their challenging environment, sharing genetic, morphological, physiological, and functional traits that demonstrate convergent evolution in response to similar environmental constraints. They can be found throughout the tropics, with major genera like Rhizophora and Avicennia present in both the Indo-West Pacific and the Atlantic, Caribbean, and Eastern Pacific realms. Mangroves are known for forming dense, often monospecific stands that create habitats for numerous terrestrial, intertidal, and marine species, stabilize shorelines, and modulate nutrient cycling and energy flow. They are an important coastal ecosystem with high net primary produc

In [38]:
memory.clear()

In [39]:
response = run_agent("What are mangroves?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("How are mangroves doing in Florida?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

print_chat_history()


🧪 User query: What are mangroves?
🔍 Detected goal: research
🤖 LLM Answer: Mangroves are a taxonomically diverse group of approximately 70 tree, shrub, and fern species that grow in anoxic and saline peaty soils on sheltered, tropical coasts. They belong to at least 25 genera and 19 families. Mangroves share genetic, morphological, physiological, and functional traits that have evolved in response to similar environmental constraints. These adaptations allow them to thrive in extreme environments characterized by high salinity, tidal variations, strong winds, high temperatures, and anaerobic tidal swamps. Mangroves can be found throughout the tropics and form dense, often monospecific stands. They provide crucial habitats for many species, stabilize shorelines, and play a significant role in nutrient cycling and energy flow.

🧪 User query: How are mangroves doing in Florida?
🔍 Detected goal: forecast
=== Top 5 Nearest Wind Station ===
Here are the top 5 stations that are most relevant o

In [40]:
response = run_agent("What is the impact of climate change on mangrove coverage?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()


🧪 User query: What is the impact of climate change on mangrove coverage?
🔍 Detected goal: research
🤖 LLM Answer: Climate change impacts the coverage of mangroves in several ways. While rising sea levels, extreme heat, high precipitation, and frequent storms pose threats to mangrove cover, climate change can also lead to an expansion of their range. Rising sea levels and warming temperatures may create more suitable habitats, allowing mangroves to expand poleward as they adapt to these changing conditions. This has been observed in places like the Gulf of Mexico, where mangroves are expanding in states like Texas, Louisiana, and Florida. However, sea-level rise and other climate change factors can also lead to the flooding and potential extinction of mangrove forests if they are unable to migrate or adapt quickly enough. Additionally, changes in precipitation patterns affect mangrove expansion, with positive effects noted in areas like Moreton Bay, Australia. Therefore, climate change c

In [41]:
print_chat_history()


🧠 Chat History:
👤 User: What are mangroves?

🤖 Assistant: Mangroves are a taxonomically diverse group of approximately 70 tree, shrub, and fern species that grow in anoxic and saline peaty soils on sheltered, tropical coasts. They belong to at least 25 genera and 19 families. Mangroves share genetic, morphological, physiological, and functional traits that have evolved in response to similar environmental constraints. These adaptations allow them to thrive in extreme environments characterized by high salinity, tidal variations, strong winds, high temperatures, and anaerobic tidal swamps. Mangroves can be found throughout the tropics and form dense, often monospecific stands. They provide crucial habitats for many species, stabilize shorelines, and play a significant role in nutrient cycling and energy flow.

👤 User: How are mangroves doing in Florida?

🤖 Assistant: Mangroves in Florida appear to be in relatively stable condition, as indicated by an NDVI prediction of 0.4832. This valu

In [42]:
response = run_agent("How are mangroves doing in Vermont?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

🧪 User query: How are mangroves doing in Vermont?
🔍 Detected goal: forecast
[resolve_gps] location=Vermont, state=VT
Chunk 2025-02-12–2025-03-02 failed: 400 Client Error: Bad Request for url: https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=20250212&end_date=20250302&product=wind&interval=h&units=english&time_zone=gmt&format=json
Chunk 2025-03-02–2025-03-20 failed: 400 Client Error: Bad Request for url: https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=20250302&end_date=20250320&product=wind&interval=h&units=english&time_zone=gmt&format=json
Chunk 2025-03-20–2025-04-07 failed: 400 Client Error: Bad Request for url: https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=20250320&end_date=20250407&product=wind&interval=h&units=english&time_zone=gmt&format=json
No valid data collected for wind.
Chunk 2025-02-12–2025-03-02 failed: 400 Client Error: Bad Request for url: https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=

In [43]:
response = run_agent("How do mangrove forests contribute to carbon sequestration, and why are they considered important in the fight against climate change?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("What adaptive strategies do mangroves employ to cope with rising sea levels and increased storm intensity?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("In what ways does the biodiversity within mangrove ecosystems enhance their resilience to climate change impacts?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("What are the main threats to mangrove ecosystems under current climate change scenarios, and what conservation strategies can be implemented to mitigate these threats?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("How does the degradation of mangrove forests affect coastal communities, both environmentally and socioeconomically?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("What role can policy and sustainable management practices play in protecting mangrove ecosystems in the context of global climate change?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()


🧪 User query: How do mangrove forests contribute to carbon sequestration, and why are they considered important in the fight against climate change?
🔍 Detected goal: research
🤖 LLM Answer: Mangrove forests are critical in carbon sequestration because they are among the most carbon-rich forests in the tropics. These ecosystems are highly efficient at carbon cycling and storage, being able to store five times more carbon per unit area than other forest ecosystems and up to three times more than other tropical forests. The role of mangroves as carbon sinks is significant because they store carbon both in their biomass and in the soil beneath them. The carbon stored in mangroves is coupled with oceanic transport mechanisms, including tidal action and currents, which helps to disperse and deposit carbon in shelf or deep water reservoirs where it can be immobilized for long periods, ranging from decades to millennia.

This carbon sequestration ability makes mangrove forests a vital tool in t

In [44]:
response = run_agent("What are the current temperature, humidity, wind speed, and precipitation levels in the coastal region adjacent to a major mangrove forest (e.g. Florida Everglades)?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("How do today’s weather forecasts in mangrove-rich regions compare to historical weather patterns, particularly in terms of storm frequency and intensity?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("Are there any active severe weather warnings or advisories for coastal areas near mangrove ecosystems, and what potential impacts might these conditions have on the mangrove health?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("What are the current sea surface temperatures and tidal forecasts near mangrove areas, and how might these factors affect the ecosystem's resilience to climate change?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

response = run_agent("How is ongoing tropical storm or cyclone activity influencing the immediate weather conditions and environmental stress on nearby mangrove forests?")
print(f'🤖 LLM Answer: {response["final_output"]}')
print()

🧪 User query: What are the current temperature, humidity, wind speed, and precipitation levels in the coastal region adjacent to a major mangrove forest (e.g. Florida Everglades)?
🔍 Detected goal: forecast
=== Top 5 Nearest Wind Station ===
1. South Port Everglades, FL
2. Virginia Key, FL
3. Vaca Key, Florida Bay, FL
4. Key West, FL
5. Fort Myers, FL

=== Top 5 Nearest Water Level Station ===
1. South Port Everglades, FL
2. Virginia Key, FL
3. Vaca Key, Florida Bay, FL
4. Key West, FL
5. NAPLES BAY, NORTH, FL

[resolve_gps] location=Florida Everglades, state=FL
[resolve_gps] Geocoded → 26.08140565, -80.23827465
[wind lags] [10.94220238095238, 8.382395833333334, 11.994285714285715, 10.828333333333333, 9.112760416666667, 12.570357142857144, 12.353630952380952, 13.849583333333333]
[water lags] [1.260499404761905, 1.8195854166666665, 1.599686904761905, 1.4838642857142856, 1.4611333333333334, 1.3415910714285715, 1.355603335318642, 1.223575]
[ndvi lags] [0.33438471777865175, 0.27073803921453

In [45]:
print_chat_history()


🧠 Chat History:
👤 User: What are mangroves?

🤖 Assistant: Mangroves are a taxonomically diverse group of approximately 70 tree, shrub, and fern species that grow in anoxic and saline peaty soils on sheltered, tropical coasts. They belong to at least 25 genera and 19 families. Mangroves share genetic, morphological, physiological, and functional traits that have evolved in response to similar environmental constraints. These adaptations allow them to thrive in extreme environments characterized by high salinity, tidal variations, strong winds, high temperatures, and anaerobic tidal swamps. Mangroves can be found throughout the tropics and form dense, often monospecific stands. They provide crucial habitats for many species, stabilize shorelines, and play a significant role in nutrient cycling and energy flow.

👤 User: How are mangroves doing in Florida?

🤖 Assistant: Mangroves in Florida appear to be in relatively stable condition, as indicated by an NDVI prediction of 0.4832. This valu